<a href="https://colab.research.google.com/github/Vitalyastiy/cheat/blob/main/202305_Interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тест

### Блок 1

1. В команде работают 11 аналитиков. На новогодних праздников ежедневно должно дежурить 2 аналитика. Скольким способами можно разделить 11 аналитиков по 2 человека. 
2. В двух урнах находится соответственно 4 и 5 белых и 6 и 3 чёрных шаров. Из каждой урны наудачу извлекается один шар, а затем из этих двух наудачу берется один. Какова вероятность, что это будет белый шар?
3. Есть 2 таблицы c 10 и 20 строк соотвественно. Сколько строк будет минимально / максимально при INNER JOIN / LEFT JOIN / OUTER JOIN / CROSS JOIN

### Блок 2 

**1. Допустим, у нас есть статистика по покупкам пользователей в приложении в таблице transactions:**

In [ ]:
| user_id | date       |
|---------|------------|
| 1       | 2021-06-01 |
| 234     | 2021-06-02 |
| 3       | 2021-06-02 |
| 1       | 2021-06-02 |
| ...     | ...        |
| 234     | 2021-09-30 |

1.1 Необходимо написать запрос, который получает количество удержанных пользователей в месяц. Данный параметр определяется как количество пользователей, которые совершили **повторную покупку в следующем месяце**.

In [ ]:

select user_id
from(
  select date_m, user_id, row_number(), over(partition by user_id order by date_m) as per_num
  from(
    select (trunc(t1.date, as "rm")) as date_m, user_id,
    from  transactions t1
    group by 1,2
  ) t
) t2
where  1=1
and per_num = 1

intesect

select user_id
from(
  select date_m, user_id, row_number(), over(partition by user_id order by date_m) as per_num
  from(
    select (trunc(t1.date, as "rm")) as date_m, user_id,
    from  transactions t1
    group by 1,2
  ) t
) t2
where  1=1
and per_num = 2




1.2 Посчитать количество отвалившихся пользователей, которые не совершили повторную покупку

**2. Допустим, у нас есть таблица transactions в таком виде, где revenue это стоимость транзакции, отрицательное значения это возврат средств:**

In [ ]:
| date       | revenue       | tr_type |
|------------|---------------|---------|         
| 2021-01-01 | -1000         | a       |
| 2021-01-01 |  1000         | b       |
| 2021-01-02 | -1100         | a       |
| 2021-01-02 |  1100         | c       |
| 2021-01-03 |  1050         | b       |
| ...        | ...           | ...     |
| 2021-09-30 |  750          | c       |

2.1 Необходиом написать запрос с использованием оконной функции, чтобы получить нарастающий итог по revenue на каждый день.

In [ ]:
select date , sum(revenue) over(order by date)
from (select date  , sum(revenue) as revenue from transactions group by 1) t1

2.2 Для каждого месяца вывести тип продукта (tr_type), который: a) принес наибольшую выручку б) по которому были наибольшие возвраты

In [ ]:
select t1.month, t1.tr_type, max_rev, max_ret
from(
  select trunc(date, "rm") as month, max(revenue) max_rev, tr_type 
  from transactions 
  where revenue>0
  group by 1,3
) t1
left join (
    select trunc(date, "rm") as month, max(revenue) max_ret, tr_type 
    from transactions 
    where revenue<0
    group by 1,3
) t2 on t1.month = t2.month


**3. Данные о погоде**

In [ ]:
+----+------------+-------------+
| id | recordDate | Temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+

Result table:
+----+
| id |
+----+
| 2  |
| 4  |
+----+
In 2015-01-02, temperature was higher than the previous day (10 -> 25).
In 2015-01-04, temperature was higher than the previous day (20 -> 30).

3.1 Необходим запрос, который позволяет вытащить id, при которых температура была больше предыдущего дня.

In [ ]:
select case when Temperature < temp_last_date then id else null end id
from (
select id, recordDate, Temperature,
lead(Temperature) over(order by recordDate) temp_last_date
from table
) t1 
where id is not null 

### Блок 3

Есть датасет *df.csv*

Структура:
* time
* user_id
* action_type
* client_platform

Необходио построить временной ряд, который отражает кол-во уникальных пользоватлей по каждому событию за каждый день

In [2]:
import pandas as pd 
df = pd.read_csv('/content/df.csv')
df.head(3)

,time,user_id,action_type,client_platform
0,2022-06-01 10:04:33,237,start_video,web
1,2022-06-01 03:33:00,693,call,android
2,2022-06-01 23:40:39,713,send_msg,android


In [ ]:
t = df.groupby(['time','action_type'], as_index = False).agg({'user_id': pd.Series.nunique})
t['date'] =  c
t1 = t.groupby(['date','action_type'], as_index = False).agg({'user_id': "sum"})
t1

In [30]:
df['date'] = pd.to_datetime(t['time']).dt.date

In [31]:
df.groupby(['date','client_platform'], as_index = False).agg({'user_id': pd.Series.nunique})

,date,client_platform,user_id
0,2022-06-01,android,38
1,2022-06-01,ios,32
2,2022-06-01,web,27
3,2022-06-02,android,31
4,2022-06-02,ios,42
...,...,...,...
166,2022-07-26,ios,35
167,2022-07-26,web,26
168,2022-07-27,android,40
169,2022-07-27,ios,29


In [ ]:
import seaborn as sns 
sns.lineplot(x= "date", y ="user_id", hue = "action_type", data = t1)

Вывести день по каждой платформе, в который DAU было максимальным

### Блок 4

Есть сервис по размещению объявлений. Вам собран датасет с некоторыми событиями из этого сервиса. Проанализируйте его и расскажите, какие продуктовые метрики из него можно посторить? Какие выводы сделать? 

Представим, что в этом сервисе нет монетизации и ее решили ввести, какие варианты ты бы предложил? На какие метрики стоит посмотреть, чтобы принять решение о монетизации? Какие метрики выбирешь для мониторинга? Опиши как собирать такие метрики. Как бы ты проводил АБ тестирование, опиши процесс. Как по выбранным метрикам можно обосновать успех или провал запуска монетизации?